In [1]:
import sys
sys.path.insert(1,'../models')
sys.path.insert(2,'../features')
import create_model
import build_features
import pandas as pd
import numpy as np
import math
import statistics
import csv
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv("../../data/processed/receiving.csv")
data.head()

,Unnamed: 0,id,Player,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,Lng,Y/Tgt,R/G,Y/G,Fmb,TD/G
0,0,AbduAm00,Ameer Abdullah,22,RB,16,9,38.0,25,65.8%,183,7.3,1,36,4.8,1.6,11.4,5.0,0.1
1,1,AbduAm00,Ameer Abdullah,23,RB,2,2,5.0,5,100.0%,57,11.4,1,18,11.4,2.5,28.5,0.0,0.5
2,2,AbduAm00,Ameer Abdullah,24,RB,14,11,35.0,25,71.4%,162,6.5,1,22,4.6,1.8,11.6,2.0,0.1
3,3,AbduAm00,Ameer Abdullah,25,RB,10,0,4.0,3,75.0%,28,9.3,0,12,7.0,0.3,2.8,1.0,0.0
4,4,AbduAm00,Ameer Abdullah,26,RB,16,0,21.0,15,71.4%,88,5.9,1,16,4.2,0.9,5.5,1.0,0.1


In [3]:
# Get all player ids
players = data.id.unique()

In [4]:
# For each receiver
offense_type = "receiving"
yards_error = []
td_error = []
non_rookies = []
yards = []
tds = []
for player in players:
    # Get their career data
    career_data = build_features.get_model_data(offense_type,player)
    if isinstance(career_data,str):
        continue
    non_rookies.append(player)
    
    # Split data into training and testing data
    numberOfYears = len(career_data.index)
    training_data = career_data.iloc[0:numberOfYears-1]
    testing_data = career_data.iloc[numberOfYears-1]
    
    # Prepare training data
    X_train = training_data[["G","Tgt","Rec","Y/R","R/G","Y/G","TD/G"]]
    y_train = training_data[["Yds","TD"]]
    
    # Prepare testing data
    X_test = testing_data[["G","Tgt","Rec","Y/R","R/G","Y/G","TD/G"]]
    X_test = np.asarray(X_test)
    X_test = X_test.reshape(-1,7)
    
    y_test = testing_data[["Yds","TD"]]
    y_test = np.asarray(y_test)
    y_test = y_test.reshape(-1,2)
    
    mdl = create_model.create_model(X_train,y_train)
    
    # Make prediction on model
    y_pred = mdl.predict(X_test)
    yards.append(y_pred[0][0])
    tds.append(y_pred[0][1])
    
    try:
        yards_error.append((abs(math.floor(round(y_pred[0][0]) - y_test[0][0]))/math.floor(y_test[0][0]))*100)
    except ZeroDivisionError:
        yards_error.append(0)
    
    try:
        td_error.append((abs(math.floor(round(y_pred[0][1]) - y_test[0][1]))/math.floor(y_test[0][1]))*100)
    except ZeroDivisionError:
        td_error.append(0)
    

In [5]:
print(statistics.mean(yards_error))

45.97720606116643


In [6]:
print(statistics.mean(td_error))

103.89639482381418


In [7]:
filename = "../../data/predictions/receiving.csv"
f = open(filename,'w')
writer = csv.writer(f)
for i in range(len(non_rookies)):
    writer.writerow([non_rookies[i],yards[i],tds[i]])

In [9]:
# Write data to csv file for further processing
filename = "../../data/predictions/error_receiving.csv"
f = open(filename,'w')
writer = csv.writer(f)
for i in range(len(non_rookies)):
    writer.writerow([non_rookies[i],yards_error[i],td_error[i]])